In [1]:
import MMGPD

In [2]:
import tiktaalik as tk

In [3]:
import numpy as np

In [4]:
myAnalysis = MMGPD.GPDAnalysis("HGAG23")

LHAPDF 6.5.4 loading /opt/homebrew/anaconda3/envs/evolution/share/LHAPDF/NNPDF40_nlo_as_01180/NNPDF40_nlo_as_01180_0000.dat
NNPDF40_nlo_as_01180 PDF set, member #0, version 1; LHAPDF ID = 331700
LHAPDF 6.5.4 loading /opt/homebrew/anaconda3/envs/evolution/share/LHAPDF/NNPDFpol11_100/NNPDFpol11_100_0000.dat
NNPDFpol11_100 PDF set, member #0, version 1; LHAPDF ID = 251000
LHAPDF 6.5.4 loading all 101 PDFs in set NNPDF40_nlo_as_01180
NNPDF40_nlo_as_01180, version 1; 101 PDF members
LHAPDF 6.5.4 loading all 101 PDFs in set NNPDFpol11_100
NNPDFpol11_100, version 1; 101 PDF members
#############################################################################
Thanks for using our analysis! The corresponding paper is: arXiv:2211.09522v2
#############################################################################




In [5]:
myAnalysis.xGPDxi("Set11", "H" , "uv", 0.25,-1,0.25)

0.2503155238948277

In [6]:
def myHu(x, t, xi):
    results = []
    for item in x:  # Iterate over elements in x
        if item > 0:
            results.append(myAnalysis.xGPDxi("Set11", "H", "uv", item, t, xi)/item)
        elif item < 0:
            results.append(myAnalysis.xGPDxi("Set11", "H", "ubar", -item, t, xi) / item)
    
    # Dynamically reshape to (len(x), 1, 1)
    return np.array(results).reshape(len(x), 1, 1)


In [7]:
def myHd(x, t, xi):
    results = []
    for item in x:  # Iterate over elements in x
        if item > 0:
            results.append(myAnalysis.xGPDxi("Set11", "H", "dv", item, t, xi)/item)
        elif item < 0:
            results.append(myAnalysis.xGPDxi("Set11", "H", "dbar", -item, t, xi) / item)
    
    # Dynamically reshape to (len(x), 1, 1)
    return np.array(results).reshape(len(x), 1, 1)


In [8]:
def myHtu(x, t, xi):
    results = []
    for item in x:  # Iterate over elements in x
        if item > 0:
            results.append(myAnalysis.xGPDxi("Set11", "Ht", "uv", item, t, xi)/item)
        elif item < 0:
            results.append(myAnalysis.xGPDxi("Set11", "Ht", "ubar", -item, t, xi) / item)
    
    # Dynamically reshape to (len(x), 1, 1)
    return np.array(results).reshape(len(x), 1, 1)


In [9]:
def myHtd(x, t, xi):
    results = []
    for item in x:  # Iterate over elements in x
        if item > 0:
            results.append(myAnalysis.xGPDxi("Set11", "Ht", "dv", item, t, xi)/item)
        elif item < 0:
            results.append(myAnalysis.xGPDxi("Set11", "Ht", "dbar", -item, t, xi) / item)
    
    # Dynamically reshape to (len(x), 1, 1)
    return np.array(results).reshape(len(x), 1, 1)


In [10]:
# Let's start with one xi value, x=0.5.
xi = 0.1
# Currently, tiktaalik requires the x grid to be spaced in a peculiar way,
# which we call a "pixelspace".
# Basically, the x values are the central values of nx intervals evenly dividing
# the domain [-1,1]. For instance, if nx=4, then [-1,1] is divided into the four
# intervals [-1,-0.5], [-0.5,0], [0,0.5] and [0.5,1]. The midpoints of these are
# -0.75, -0.25, 0.25 and 0.75. These four midpoints are the pixelspace values.

# We can make a pixelspace using the tiktaalik package.
# Let's make one with 20 points to start.
# Note that tiktaalik *requires* the number of x points to be even.
nx = 100
x = tk.matrices.pixelspace(nx)

In [11]:
# Let's create a model GPD we want to evolve
# We'll use the tk-supplied GK model code.
# However, as long as we make sure to use a pixelspace for x,
# we can use any GPD function.
# Let's create a non-singlet combination so it evolves by itself.
# The Hu-minus distribution will do.
t = -1
umin0 = myHu(x, t, xi) + myHu(-x, t, xi)


In [12]:
x[0]

-0.99

In [13]:
type(umin0)

numpy.ndarray

In [14]:
# Now, we want to make an evolution matrix.
# Before we can do this, we need to call the routines to initialize
# the kernel and evolution matrices that tiktaalik's Fortran code holds in memory.
# (These are held in memory to avoid needing to frequent rebuildings.
# I'll explain this in a little bit.)
# The kernels need to be initialized with info about the number of x points
# and the specific collection of xi values we're using.
tk.matrices.initialize_kernels(nx=nx, xi=xi)


In [15]:
# Next, we need to initialize the evolution matrices.
# We need to come up with some Q2 array.
# The GK model is designed to work at Q2 = 4 GeV**2.
# Let's say we want to evolve to 5, 6 and 7 GeV**2.
# Then we need an array with all four values:
Q2 = np.array([4, 5, 20, 100]) # units of GeV**2
tk.matrices.initialize_evolution_matrices(Q2)

In [16]:
# Now we're allowed to get the evolution matrices!
# Let's get the non-singlet, helicity-independent one.
M_NS = tk.matrices.matrix_VNS()


In [17]:
# The matrix will have dimensions (nx, nx, nxi, nQ2).
# Let's check!
print("Shape of M_NS: ", M_NS.shape)
# Note that the GPD we're using has a shape (nx,nxi,nt).
print("Shape of umin0: ", umin0.shape)
# To get an evolved GPD, we use np.einsum
umin_evo = np.einsum('ijkl,jk...->ik...l', M_NS, umin0)
# The ellipsis makes sure any extra dimensions (t in this case)
# are placed in-between the (x,xi) and (Q2) dimensions.
print("Shape of umin_evo: ", umin_evo.shape)

# We can reuse the same matrix to evolve different non-singlet distributions.
dmin0 = tk.model.Hd(x, xi, t=0) + tk.model.Hd(-x, xi, t=0)
dmin_evo = np.einsum('ijkl,jk...->ik...l', M_NS, dmin0)

# We can also do singlet evolution, but for this we need to concatenate
# the singlet quark and gluon arrays.
sing0 = tk.model.H_singlet(x, xi, t=0)
glue0 = tk.model.Hg(x, xi, t=0)
sg0 = np.concatenate((sing0, glue0), axis=0)
# We grab the singlet-gluon matrix, which has dimensions (2*nx, 2*nx, nxi, nQ2).
M_SG = tk.matrices.matrix_VSG()
print("Shape of M_SG: ", M_SG.shape)
print("Shape of sg0: ", sg0.shape)
# The dimensions match, so we can evolve as usual.
sg_evo = np.einsum('ijkl,jk...->ik...l', M_SG, sg0)
# And the singlet and gluon GPDs can be grabbed by slicing the array
sing_evo = sg_evo[0:nx,...]
glue_evo = sg_evo[nx:2*nx,...]

Shape of M_NS:  (100, 100, 1, 4)
Shape of umin0:  (100, 1, 1)
Shape of umin_evo:  (100, 1, 1, 4)
Shape of M_SG:  (200, 200, 1, 4)
Shape of sg0:  (200, 1, 1)


In [18]:
ioi = tk.model.Hu(x, xi, t=-1)

In [19]:
umin0

array([[[-4.73758920e-06]],

       [[-2.74251211e-05]],

       [[-2.56972028e-05]],

       [[ 7.00218075e-05]],

       [[ 3.54438179e-04]],

       [[ 9.53673617e-04]],

       [[ 2.02770933e-03]],

       [[ 3.77814566e-03]],

       [[ 6.44797143e-03]],

       [[ 1.03158044e-02]],

       [[ 1.56878051e-02]],

       [[ 2.28748495e-02]],

       [[ 3.21808110e-02]],

       [[ 4.38914476e-02]],

       [[ 5.82737715e-02]],

       [[ 7.55778584e-02]],

       [[ 9.60317672e-02]],

       [[ 1.19833315e-01]],

       [[ 1.47140990e-01]],

       [[ 1.78063223e-01]],

       [[ 2.12649691e-01]],

       [[ 2.50888602e-01]],

       [[ 2.92694725e-01]],

       [[ 3.37914802e-01]],

       [[ 3.86306563e-01]],

       [[ 4.37542405e-01]],

       [[ 4.91173812e-01]],

       [[ 5.46618320e-01]],

       [[ 6.03131062e-01]],

       [[ 6.59785412e-01]],

       [[ 7.15489510e-01]],

       [[ 7.69002622e-01]],

       [[ 8.19000477e-01]],

       [[ 8.64125157e-01]],

       [[ 9.03

In [20]:
umin_evo

array([[[[-4.73758920e-06, -4.11080135e-06, -1.96184201e-06,
          -1.01654900e-06]]],


       [[[-2.74251211e-05, -2.46090105e-05, -1.37267409e-05,
          -7.92675672e-06]]],


       [[[-2.56972028e-05, -2.42718849e-05, -1.71121829e-05,
          -1.18048062e-05]]],


       [[[ 7.00218075e-05,  6.04025304e-05,  2.63756709e-05,
           1.11023360e-05]]],


       [[[ 3.54438179e-04,  3.16965244e-04,  1.73715163e-04,
           9.85011651e-05]]],


       [[[ 9.53673617e-04,  8.62880026e-04,  5.04591866e-04,
           3.05325694e-04]]],


       [[[ 2.02770933e-03,  1.84809437e-03,  1.12425488e-03,
           7.06497699e-04]]],


       [[[ 3.77814566e-03,  3.46237516e-03,  2.16910401e-03,
           1.40138757e-03]]],


       [[[ 6.44797143e-03,  5.93570707e-03,  3.80912684e-03,
           2.51690346e-03]]],


       [[[ 1.03158044e-02,  9.53347462e-03,  6.24678840e-03,
           4.20830853e-03]]],


       [[[ 1.56878051e-02,  1.45493717e-02,  9.71406268e-03,
         

In [21]:
x


array([-0.99, -0.97, -0.95, -0.93, -0.91, -0.89, -0.87, -0.85, -0.83,
       -0.81, -0.79, -0.77, -0.75, -0.73, -0.71, -0.69, -0.67, -0.65,
       -0.63, -0.61, -0.59, -0.57, -0.55, -0.53, -0.51, -0.49, -0.47,
       -0.45, -0.43, -0.41, -0.39, -0.37, -0.35, -0.33, -0.31, -0.29,
       -0.27, -0.25, -0.23, -0.21, -0.19, -0.17, -0.15, -0.13, -0.11,
       -0.09, -0.07, -0.05, -0.03, -0.01,  0.01,  0.03,  0.05,  0.07,
        0.09,  0.11,  0.13,  0.15,  0.17,  0.19,  0.21,  0.23,  0.25,
        0.27,  0.29,  0.31,  0.33,  0.35,  0.37,  0.39,  0.41,  0.43,
        0.45,  0.47,  0.49,  0.51,  0.53,  0.55,  0.57,  0.59,  0.61,
        0.63,  0.65,  0.67,  0.69,  0.71,  0.73,  0.75,  0.77,  0.79,
        0.81,  0.83,  0.85,  0.87,  0.89,  0.91,  0.93,  0.95,  0.97,
        0.99])